In [ ]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle
import signac

from pympler import asizeof

from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

#Ignore inconcistent version warning
import warnings
# from sklearn.exceptions import InconsistentVersionWarning
# warnings.filterwarnings(action='ignore', category=InconsistentVersionWarning)

In [ ]:
#From signac
import signac
meth_name_val_list = [1, 2, 3, 4, 5, 6]
save_csv = False

criteria_dict = {"cs_name_val" : 1,
                 "ep_enum_val": {"$gte": 1},
                 "meth_name_val": {"$in": meth_name_val_list}}

criteria_dict = {"cs_name_val" : 1,
                 "ep_enum_val": 1,
                 "meth_name_val": {"$in": meth_name_val_list}}


In [ ]:
project = signac.get_project()
save_csv = False
save_figs = False
analyzer = General_Analysis(criteria_dict, project, save_csv)
plotters = Plotters(analyzer, save_figs)
# df_all_jobs, job_list, theta_true = analyzer.get_df_all_jobs()
df_best, job_list_best = analyzer.get_best_data()

# for i in range(len(job_list_best)): 
#     print(job_list_best[i].id)
df_best

In [ ]:
#Make Heat maps for all pairs
z_choices = ["sse_sim", "sse_mean", "sse_var", "ei"]
levels = [100,100,100,100]

#Loop over best jobs
for i in range(len(job_list_best)):   
    #Get jobs, runs, and iters to examine
    job = job_list_best[i]
    run_num = df_best["Run Number"].iloc[i]
    bo_iter = df_best["BO Iter"].iloc[i]
    
    #Back out number of parameters
    string_val = df_best["Theta Min Obj"].iloc[0]
    try:
        numbers = [float(num) for num in string_val.replace('[', '').replace(']', '').split()]
    except:
        numbers = [float(num) for num in string_val]
        
    #Create list of parameter pair combinations
    dim_theta = len(np.array(numbers).reshape(-1, 1))
    dim_list = np.linspace(0, dim_theta-1, dim_theta)
    pairs = len((list(combinations(dim_list, 2))))
    
    #Loop over parameter pairs
    for pair in range(pairs):
        plotters.plot_hms_gp_compare(job, run_num, bo_iter, pair, z_choices, levels)

In [ ]:
levels = [100, 100, 100, 100]
pair = 0
z_choice = ["sse_sim", "sse_mean", "sse_var", "ei"]
plotters.plot_hms_gp_compare(job_list_best[0], df_best["Run Number"].iloc[0], df_best["BO Iter"].iloc[0], 
                             pair, z_choice, levels, log_data = False, title = None)

In [ ]:
# title = "Heat Map Pair " + z_choice
levels = [100, 100, 100, 100, 100, 100]
pair = 0
z_choice = "sse_mean"
plotters.plot_hms_all_methods(pair, z_choice, levels, log_data = False, title = None)

In [ ]:
plotters.plot_objs_all_methods(["min_sse", "sse"])

In [ ]:
plotters.plot_hypers(job_list_best[0], title = None)

In [ ]:
plotters.plot_one_obj_all_methods("min_sse")

In [ ]:
plotters.plot_thetas(job_list_best[0], "min_sse", title = None)